In [75]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import json
import jsonlines
import re
import numpy as np

In [76]:
proy_info_data=pd.read_csv("data/base_juguete - proyecto_info.tsv", sep='\t',na_values=None, keep_default_na=False)
proy_talentos_data=pd.read_csv("data/base_juguete - proyecto_talentos.tsv", sep='\t',na_values=None, keep_default_na=False)

In [77]:
#proy_info_data

In [78]:
#ENTIDADES

class info(object):
    uid=""
    numero=""
    pais=""
    nombre=""
    giro=""
    proyecto=""
    descproyecto=""
    objetivo=""
    entregable=""
    ubicacion=""
    inicio=""
    fin=""
    pago=""


class talentos(object):
    uid=""
    grados=""
    experiencia=""
    funciones=""
    idiomas=""
    almamater=""
    habilidades=""

    def __init__(self, grados, experiencia, funciones, idiomas, almamater, habilidades):
        self.grados=grados
        self.experiencia=experiencia
        self.funciones=funciones
        self.idiomas=idiomas
        self.almamater=almamater
        self.habilidades=habilidades

        
    def __json__(self):
        return {'PRO_TITULO':self.grados, 'PRO_EXP':self.experiencia, 'PRO_FUNCIONES':self.funciones,'PRO_IDIOMAS':self.idiomas, 
               'PRO_INST':self.almamater, 'PRO_HABILIDADES':self.habilidades}

class proyecto(object):
    uid=""
    numero=""
    pais=""
    nombre=""
    giro=""
    proyecto=""
    descproyecto=""
    objetivo=""
    entregable=""
    ubicacion=""
    inicio=""
    fin=""
    pago=""
    talentos=[]
    
    def __init__(self, uid, numero, pais, nombre, giro, proyecto, descproyecto, objetivo, entregable, ubicacion, inicio, fin, pago):
        self.uid = uid
        self.numero = numero
        self.pais = pais
        self.nombre = nombre
        self.giro = giro
        self.proyecto = proyecto
        self.descproyecto = descproyecto
        self.objetivo = objetivo
        self.entregable = entregable
        self.ubicacion = ubicacion
        self.inicio = inicio
        self.fin = fin
        self.pago = pago
        
        
    def __json__(self):
        return {'uid': self.uid, 'numero': self.numero, 'pais':self.pais ,'nombre':self.nombre,
                'giro':self.giro, 'proyecto':self.proyecto, 'descproyecto':self.descproyecto,
                'objetivo':self.objetivo,'entregable':self.entregable, 'ubicacion':self.ubicacion,'inicio':self.inicio,
                'fin':self.fin,'pago':self.pago,'talentos':self.talentos}
    
    
class MyEncoder(json.JSONEncoder):
    """
    JSONEncoder subclass that leverages an object's `__json__()` method,
    if available, to obtain its default JSON representation. 

    """
    def default(self, obj):
        if hasattr(obj, '__json__'):
            return obj.__json__()
        return json.JSONEncoder.default(self, obj)


In [79]:
'''
HELPERS : 
'''
def parse_doc_list_plain(doc):        
    '''Preprocesing method, this code dont remove the character with graphic accent.'''
    red = ur'[^,a-z\u00E7\u00F1\u00E1\u00E9\u00ED\u00F3\u00FA\u00E0\u00E8\u00EC\u00F2\u00F9\u00E4\u00EB\u00EF\u00F6\u00FC\u00E2\u00EA\u00EE\u00F4\u00FB ]'
    ##sandbox    
    
    art_vect = re.sub(r' +',' ', re.sub(red,'',re.sub(ur'\u201c',' ',re.sub('\n',' ',doc).lower()))) # Remove the most of the symbols. 
    art_vect = re.sub(',+',',',art_vect) # 
    art_vect = re.sub(', ',',',art_vect) # ,\s\+   remove space after comma, it could be improved
    art_vect = re.sub('^,','',art_vect) # ,\s\+   remove space after comma, it could be improved    
    art_vect = re.sub(r' ','_',art_vect) # Replace the space by underscore.
    art_vect = re.sub(r',$','',art_vect)  # Remove the last comma character. just in case.
    return art_vect

In [80]:
test_text = u',La mansion puprpura de Zimbagüe es la casa, que,,,hemos creado para tí, donde,...'

In [81]:
parse_doc_list_plain(test_text)

u'la_mansion_puprpura_de_zimbag\xfce_es_la_casa,que,hemos_creado_para_t\xed,donde'

In [82]:
proyectos_data = []

for i in proy_info_data.itertuples():
    data = proyecto(i.uid,i.numero,i.pais,i.nombre,
                    i.giro,i.descproyecto,i.objetivo,i.proyecto,i.entregable,
                    i.ubicacion,i.inicio,i.fin,i.pago)
    match_talentos = proy_talentos_data['uid'] == data.uid
    data.talentos = [talentos(grados=ta.grados,experiencia=ta.experiencia,funciones=ta.funciones,
                              idiomas=ta.idiomas,almamater=ta.almamater,habilidades=ta.habilidades) 
                     for ta in proy_talentos_data[match_talentos].itertuples() ]
    #match_talentos = proy_talentos_data['uid'] == data.uid
    proyectos_data.append(json.dumps(data,cls=MyEncoder))

### PASO 0 : Preprocesamiento

In [83]:
jlArray = []
for item in jsonlines.Reader(proyectos_data).iter():
    preproc = {}    
    preproc[u'PRO_ID'] = item[u'numero']
    preproc[u'uid'] = item[u'uid']
    preproc[u'PRO_PAIS'] = item[u'pais']
    preproc[u'PRO_NOMBRE'] = item[u'nombre']
    preproc[u'PRO_PROY'] = item[u'proyecto']
    preproc[u'PRO_DESC'] = item[u'descproyecto']
    preproc[u'PRO_OBJ'] = item[u'objetivo']
    preproc[u'PRO_ENTR'] = item[u'entregable']
    preproc[u'PRO_UBIC'] = item[u'ubicacion']
    preproc[u'PRO_TITULO'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'PRO_TITULO']) for cur in item['talentos'] if cur != 'NaN']))
    preproc[u'PRO_EXP'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'PRO_EXP']) for cur in item['talentos'] if cur != 'NaN']))
    preproc[u'PRO_FUNCIONES'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'PRO_FUNCIONES']) for cur in item['talentos'] if cur != 'NaN']))
    preproc[u'PRO_IDIOMAS'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'PRO_IDIOMAS']) for cur in item['talentos'] if cur != 'NaN']))
    preproc[u'PRO_INST'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'PRO_INST']) for cur in item['talentos'] if cur != 'NaN']))
    preproc[u'PRO_HABILIDADES'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'PRO_HABILIDADES']) for cur in item['talentos'] if cur != 'NaN']))
    jlArray.append(preproc)

In [84]:
dfproyectos = pd.DataFrame(jlArray)

In [85]:
dfproyectos.to_csv('proyectos.csv')